In [ ]:
import os
import cv2
import pandas as pd
from tqdm import tqdm
from matplotlib import pyplot as plt
import numpy as np

RESIZE = 512
IMG_QUALITY = 95
DEBUG = False

In [ ]:
import math
import tensorflow as tf
import matplotlib.pyplot as plt

def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def serialize_example(feature0, feature1, feature2):
  feature = {
      'image': _bytes_feature(feature0),
      'id': _bytes_feature(feature1),
      'class': _int64_feature(feature2)
  }
  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto.SerializeToString()

In [ ]:
%cd /kaggle/working

base_path = '/kaggle/working'
train_videos_path = "/kaggle/input/nfl-impact-detection/train"

df_labels = pd.read_csv("/kaggle/input/nfl-impact-detection/train_labels.csv")
df_labels['impact'] = np.nan_to_num(df_labels['impact'].values)
content_trainlist = ""
content_testlist  = ""

for video_index, video_name in enumerate(os.listdir(train_videos_path)):
    print(video_index)
    cap = cv2.VideoCapture(os.path.join(train_videos_path,video_name))
    if (cap.isOpened() == False):
      print("Unable to read camera feed")
    frame_no = 0
    
    tf_filename = f'{video_name}.tfrec'
    with tf.io.TFRecordWriter(tf_filename) as wf:
        while(True):
            frame_no += 1
            ret, frame = cap.read()
            if not ret:
                break
            
            frame_height, frame_width,_ = frame.shape

            content = ""
            df = df_labels.query(f"frame=={frame_no}")
            df = df[df.video==video_name]
            
            img = cv2.resize(frame, (RESIZE, RESIZE))
            # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  -> Fix:20201121
            
            img = cv2.imencode('.jpg', img, (cv2.IMWRITE_JPEG_QUALITY, IMG_QUALITY))[1].tostring()
            img_name = str.encode(f"{video_name}_{frame_no}")
            target = int(df.impact.max())
            example = serialize_example(img, img_name, target)
            wf.write(example)
    cap.release()
#     if video_index > 4:
#         break

### image checking

In [ ]:
import numpy as np

def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    return image

def parse_example(example):
    LABELED_TFREC_FORMAT = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'id': tf.io.FixedLenFeature([], tf.string),
        'class': tf.io.FixedLenFeature([], tf.int64)
    }
    
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = example['id']
    target = example['class']
    return image, label, target

def display_one(image, title, target, subplot, red=False, titlesize=16):
    plt.subplot(*subplot)
    plt.axis('off')
    plt.imshow(image)
    plt.title(f'{title}: {target}')
    return (subplot[0], subplot[1], subplot[2]+1)

def display_batch_of_images(databatch):
    images, labels, targets = databatch
    images = images.numpy()
    labels = labels.numpy()
    targets = targets.numpy()
    if labels is None:
        labels = [None for _ in enumerate(images)]
    if targets is None:
        targets = [None for _ in enumerate(targets)]
        
    # auto-squaring: this will drop data that does not fit into square or square-ish rectangle
    rows = int(math.sqrt(len(images)))
    cols = len(images)//rows
        
    # size and spacing
    FIGSIZE = 13.0
    SPACING = 0.2
    subplot=(rows, cols, 1)
    if rows < cols:
        plt.figure(figsize=(FIGSIZE,FIGSIZE/cols*rows))
    else:
        plt.figure(figsize=(FIGSIZE/rows*cols,FIGSIZE))
    
    # display
    for i, (image, label, target) in enumerate(zip(images[:rows*cols], labels[:rows*cols], targets[:rows*cols])):
        title = label
        title = title.decode('utf-8')
        correct = True
        dynamic_titlesize = FIGSIZE*SPACING/max(rows,cols)*40+3 # magic formula tested to work from 1x1 to 10x10 images
        subplot = display_one(image, title, target, subplot, not correct, titlesize=dynamic_titlesize)
    
    #layout
    plt.tight_layout()
    if label is None and predictions is None:
        plt.subplots_adjust(wspace=0.2, hspace=0.2)
    else:
        plt.subplots_adjust(wspace=SPACING, hspace=SPACING)
    plt.show()

In [ ]:
resize_file = "58106_002918_Sideline.mp4.tfrec"
dataset = tf.data.TFRecordDataset([resize_file]).map(parse_example).batch(25)
data = iter(dataset)
display_batch_of_images(next(data))